<a href="https://colab.research.google.com/github/Sahar-Sheikhi/Multilingual_Translation/blob/colab-rendering/translation_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Define the path to your dataset folder
data_path = "/content/drive/MyDrive/translation_data/"

# Load datasets directly from Google Drive
import pandas as pd

train_df = pd.read_csv(data_path + "100_train_data.csv")
val_df = pd.read_csv(data_path + "10_val_data.csv")
test_df = pd.read_csv(data_path + "10_test_data.csv")


# Disable wandb to train free

In [4]:
!pip uninstall -y wandb



Found existing installation: wandb 0.19.9
Uninstalling wandb-0.19.9:
  Successfully uninstalled wandb-0.19.9


In [1]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/translation_data/100_train_data.csv')
print(df.head())


                                   English  \
0                            TRAFO 440-230   
1                          TIMER BOARD CPL   
2                             FOOT H=150MM   
3                     DOUBLE SOLNOID VALVE   
4  FULL IND,CUPB,H2,4ZONE,2S,1000X900X700H   

                                    Italian  
0                             TRASFORMATORE  
1                          SCHEDA TIMER CPL  
2                             PIEDE H=150MM  
3                     ELETTROVALVOLA DOPPIA  
4  INDUZ,AMPIA SUP,4 ZONE+VANO ARM,,2L,H700  


In [6]:
!pip install transformers datasets sentencepiece sacrebleu evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install datasets

In [3]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict


# Load the datasets
train_df = pd.read_csv('/content/drive/MyDrive/translation_data/100_train_data.csv')
val_df = pd.read_csv('/content/drive/MyDrive/translation_data/10_val_data.csv')
test_df = pd.read_csv('/content/drive/MyDrive/translation_data/10_test_data.csv')

# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Reduce dataset
train_dataset = train_dataset.select(range(min(30, len(train_dataset))))
val_dataset = val_dataset.select(range(min(5, len(val_dataset))))
test_dataset = test_dataset.select(range(min(5, len(test_dataset))))

# Verify the new sizes
print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")


from transformers import MBart50TokenizerFast

model_checkpoint = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="it_IT")

max_length = 128

def tokenize_data(example):
    inputs = tokenizer(example['English'], max_length=max_length, truncation=True, padding="max_length")
    targets = tokenizer(example['Italian'], max_length=max_length, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_data, batched=True, remove_columns=['English', 'Italian'])
tokenized_val = val_dataset.map(tokenize_data, batched=True, remove_columns=['English', 'Italian'])
tokenized_test = test_dataset.map(tokenize_data, batched=True, remove_columns=['English', 'Italian'])




Train size: 30
Validation size: 5
Test size: 5


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

# Setup Data Collator and Metrics

In [9]:
! pip install evaluate

In [10]:
!pip install sacrebleu

In [4]:
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_checkpoint, padding=True)

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    return {"bleu": result["score"]}


#Initialize the mBART Model

In [12]:
from transformers import MBartForConditionalGeneration

model = MBartForConditionalGeneration.from_pretrained(model_checkpoint)


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

#Define Training Arguments

In [13]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
       output_dir="mbart-eng-it-domain-specific",
       eval_strategy="epoch",
       save_strategy="epoch",
       learning_rate=2e-5,
       per_device_train_batch_size=1,  #  batch size
       per_device_eval_batch_size=1,  #  batch size
       num_train_epochs=3,
       weight_decay=0.01,
       predict_with_generate=True,
       save_total_limit=2,
       fp16=False,
       report_to=None
   )

# Initialize the Trainer

In [14]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


<ipython-input-14-cd3df47410e1>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


#Start training

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss


#Evaluate the model

In [ ]:
eval_results = trainer.evaluate(tokenized_test)
print(f"BLEU score on test set: {eval_results['eval_bleu']:.2f}")


BLEU score on test set: 45.18


# Save model and tokenizer

In [ ]:
trainer.save_model("mbart-eng-it-trained")
tokenizer.save_pretrained("mbart-eng-it-trained")


('mbart-eng-it-trained/tokenizer_config.json',
 'mbart-eng-it-trained/special_tokens_map.json',
 'mbart-eng-it-trained/sentencepiece.bpe.model',
 'mbart-eng-it-trained/added_tokens.json',
 'mbart-eng-it-trained/tokenizer.json')

#Inference - Testing translations

In [ ]:
from transformers import pipeline

translator = pipeline(
    "translation",
    model="mbart-eng-it-trained",
    tokenizer="mbart-eng-it-trained",
    src_lang="en_XX",
    tgt_lang="it_IT"
)

sentence = "This is a domain-specific sentence for translation."
translated_sentence = translator(sentence)[0]['translation_text']
print(f"Translated sentence: {translated_sentence}")


# Create a Translator Pipeline

In [8]:
from transformers import pipeline

translator_pipeline = pipeline(
    "translation",
    model="/content/drive/MyDrive/mbart-eng-it-trained", # path to the saved model on  Google Drive
    tokenizer="/content/drive/MyDrive/mbart-eng-it-trained", # path to the saved tokenizer on Google Drive
    src_lang="en_XX",
    tgt_lang="it_IT"
)

Device set to use cpu


# VECTORDB

In [5]:
!pip install faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 47.9 MB/s eta 0:00:00


# Prepare a Knowledge Base and Vector Index


In [9]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

reference_texts = [
    "A clinical diagnosis requires a comprehensive patient history.",
    "In legal terms, liability refers to responsibility.",
    "GDP stands for gross domestic product.",
    "Molarity is a unit of concentration in chemistry.",
]

# Load embedder and build vector index
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(reference_texts, convert_to_numpy=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

doc_store = {i: text for i, text in enumerate(reference_texts)}


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Define a Retriever Function

In [10]:
def retrieve_context(query, top_k=2):
    query_vec = embedder.encode([query])
    _, indices = index.search(np.array(query_vec), top_k)
    return [doc_store[i] for i in indices[0]]


# Define a RAG-Style Translator Wrapper

In [11]:
def rag_translate(input_text, translator_pipeline, context_k=2):
    context = retrieve_context(input_text, top_k=context_k)
    combined_input = " ".join(context) + " " + input_text
    translation = translator_pipeline(combined_input)[0]['translation_text']
    return translation


# Test VDB

In [12]:
query = "What is the meaning of molarity in a lab test?"
translated = rag_translate(query, translator_pipeline)
print("RAG Translation:", translated)


RAG Translation: La molarità è un'unità di concentrazione in chimica. Una diagnosi clinica richiede una storia completa del paziente. Che significa la molarità in un test di laboratorio?


#  Save & Load VectorDB (FAISS)

In [13]:
faiss.write_index(index, "/content/drive/MyDrive/context_index.faiss")

In [15]:
index = faiss.read_index("/content/drive/MyDrive/context_index.faiss")
